# ready 

In [1]:
from handler_raw_complaints import create_basic_complaints 

from handler_GIS_demographic import build_GIS_demographic_data
from handler_GIS_buslines_jerusalem import make_jlm_metro_routes
from handler_rishui import rishui_resolutions, passengers_resolutions, ebitzua_resolutions
from handler_socioeconomic_ranking import calculate_socio_econimoc_lines
from handler_service_areas import create_service_areas_resolutions
from handler_complaints import complaints_resolutions
from general_functions import prepare_data_to_directness_analysis, create_directness, to_df
from handler_merger_export import merge_and_export


## use arcgis pro kernel

In [2]:
sample=create_basic_complaints()
sample.head()

,ticketnumber,subjectname,incident_date,makat,routeid_direction,hour,words_to_include,words_to_exclude,relevant
0,CAS-311697-Y1N6J3,הוספת תחנה,2022-01-01,11616-2-0,11616-2,NaN,None,NaN,None
1,CAS-311793-N2B7P2,ריח לא נעים בקרון,2022-01-02,-1--1--1,-1--1,NaN,1,NaN,None
2,CAS-311982-W1T4F6,איחור מעל 10 דקות,2022-01-02,12180-2-0,12180-2,07:05,1,NaN,1
3,CAS-311994-Y3P4V2,אי ביצוע,2022-01-02,13247-1-0,13247-1,18:07,1,NaN,1
4,CAS-312006-M1J7C3,הנהג לא עצר לנוסע שהמתין בתחנה,2022-01-02,10415-2-0,10415-2,22:25,1,NaN,1


In [3]:
# process raw complaints, not to regular run
# create_basic_complaints()

# add geographic locations to demofraphic table
build_GIS_demographic_data()

#create geographic layer of bus lines in research area
bus_lines_jlm=make_jlm_metro_routes()

# add license details for each bus line in each resolution\
#  (line-direction-alternative-day period,line-direction-alternative,line-direction)
rishui_res12, rishui_res3=rishui_resolutions(bus_lines_jlm)

#calculate socio economic ranking per bus line
socio_eco_res_12, socio_eco_res3=calculate_socio_econimoc_lines(bus_lines_jlm)

#determine wether bus line serves west bank settlements, west metropolitan area, city of jerusalem, or east jerusalem
service_areas_res12, service_areas_res3=create_service_areas_resolutions(rishui_res12)

#calculate number of passengers per bus line
passengers_res12, passengers_res3=passengers_resolutions()

#calculate percent of non-execution trips per bus line
ebitzua_res1, ebitzua_res2, ebitzua_res3=ebitzua_resolutions()

# sum number of complaints per bus line
complaints_res1,complaints_res2, complaints_res3=complaints_resolutions()

#calculate direcntess mesurement per bus line (the compelete analisys was made in a different file)
prepare_data_to_directness_analysis()



statistical areas with demographic date is printed
relevant_buses_layer created
rishui data res12 is printed
rishui data res3 is printed
sosio economic ranking printed
service areas are printed
passengers numbers per line are printed
e-bitua data printed
scomplaints data printed
pre directness analysis complete


# switch to osmnx kernel (undo comment)

In [ ]:
# create_directness_measurement()

# switch back to arcgis pro kernel

In [4]:
directness_res12, directness_res3=create_directness()

#merge dfs based on resolution, and export to csvs
res1_dfs=[rishui_res12[['makat', 'type']],passengers_res12,directness_res12[['route_desc', 'directness']],
         socio_eco_res_12 ,service_areas_res12]


res2_dfs=[rishui_res12[['makat', 'type']],passengers_res12,ebitzua_res2,socio_eco_res_12,
        directness_res12[['route_desc', 'directness']],service_areas_res12, complaints_res2]

res3_dfs=[rishui_res3[['routeid_direction', 'type']],passengers_res3,ebitzua_res3,
            directness_res3,service_areas_res3,socio_eco_res3, complaints_res3]


r1, r2, r3=merge_and_export(res1_dfs,res2_dfs,res3_dfs,ebitzua_res1,complaints_res1)

res1 exported
res2 exported
res3 exported


In [5]:
r1[r1['makat']=='15019-1-א']

,makat,service_types,passengersnumber_thousands,Directness_measurements,socioeco_2019,updated_ultraorthodox_percent,Settlements_lines,West_metropolin_lines,East_JLM_lines,InJerusalem_lines,day_period,number_of_complaints,percent_problematic_trips,percent_problematic_trips
123,15019-1-א,3,16.311,0.8,2.0,22.86,0,0,0,1,3,0.0,0.0,0.0
124,15019-1-א,3,16.311,0.8,2.0,22.86,0,0,0,1,1,1.0,0.0,0.0
125,15019-1-א,3,16.311,0.8,2.0,22.86,0,0,0,1,2,0.0,0.0,0.0


In [6]:
r2[r2['makat']=='15019-1-א']

,makat,service_types,passengersnumber_thousands,percent_problematic_trips,socioeco_2019,updated_ultraorthodox_percent,Directness_measurements,Settlements_lines,West_metropolin_lines,East_JLM_lines,InJerusalem_lines,number_of_complaints
43,15019-1-א,3,16.311,0.0,2.0,22.86,0.8,0.0,0.0,0.0,1.0,1.0


In [7]:
r3[r3['routeid_direction']=='15019-1']

,routeid_direction,service_types,passengersnumber_thousands,percent_problematic_trips,Directness_measurements,Settlements_lines,West_metropolin_lines,East_JLM_lines,InJerusalem_lines,socioeco_2019,updated_ultraorthodox_percent,number_of_complaints
416,15019-1,3,28.226,0.0,0.786667,0.0,0.0,0.0,1.0,2.0,21.83,1.0


In [8]:
r1['percent_problematic_trips']

,percent_problematic_trips,percent_problematic_trips
0,0.0,0.0
1,0.0,0.0
2,0.0,0.0
3,0.0,0.0
4,0.0,0.0
...,...,...
1942,0.0,0.0
1943,0.0,0.0
1944,0.0,0.0
1945,0.0,0.0
